In [1]:
import importlib, pkg_resources
importlib.reload(pkg_resources)

<module 'pkg_resources' from '/home/pratzz/Desktop/open_source/github/GSoC_2021_QMLHEP_Tasks/quantum_env/lib/python3.8/site-packages/pkg_resources/__init__.py'>

In [3]:
! pip install tensorflow cirq sympy numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 619.4 kB/s eta 0:00:00m eta 0:00:010:00:15
  Using cached cirq-0.14.1-py3-none-any.whl (7.8 kB)
  Using cached libclang-13.0.0-py2.py3-none-manylinux1_x86_64.whl (14.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 625.2 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 1.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 336.8 kB/s eta 0:00:001m916.3 kB/s eta 0:00:01
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py) ... done
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any

  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached importlib_metadata-4.11.3-py3-none-any.whl (18 kB)
  Using cached lark-0.11.3-py2.py3-none-any.whl
  Using cached rpcq-3.10.0-py3-none-any.whl
  Using cached retry-0.9.2-py2.py3-none-any.whl (8.0 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 KB 166.9 kB/s eta 0:00:00 kB/s eta 0:00:01
  Using cached ruamel.yaml-0.17.21-py3-none-any.whl (109 kB)
  Using cached python_rapidjson-1.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached msgpack-0.6.2-cp38-cp38-linux_x86_64.whl
  Using cached ruamel.yaml.clib-0.2.6-cp38-cp38-manylinux1_x86_64.whl (570 kB)
Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
  Running setup.py install for termcolor ... done
  

In [4]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

2022-04-19 21:40:26.107804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 21:40:26.107828: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-19 21:40:45.424063: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-19 21:40:45.424133: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vivobook): /proc/driver/nvidia/version does not exist
2022-04-19 21:40:45.424748: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [6]:
import random
from itertools import product

In [9]:
def layer(qs, params):
    circ = cirq.Circuit()
    for i in reversed(range(len(qs)-1)):
        circ += cirq.CNOT(qs[i], qs[i+1])
    for i in range(len(qs)):
        circ += cirq.ry(params[2*i]).on(qs[i])
        circ += cirq.rz(params[2*i + 1]).on(qs[i])
    return circ

In [10]:
def make_circuit(qs, state, latent, params, depth, swap_qubit, reference_qubits):
    c = cirq.Circuit()
    enc_params = params[:len(params) // 2]
    dec_params = params[len(params) // 2:]
    for i in range(depth):
        c += layer(qs[:state], enc_params[2 * i * state:2 * (i + 1) * state])
    for i in range(depth):
        c += layer(qs[state - latent:], dec_params[2 * i * state:2 * (i + 1) * state])
    # SWAP Test
    c += cirq.H(swap_qubit)
    for i, j in product(range(state), range(state - latent, len(qs))):
        #c += cirq.CSWAP(swap_qubit, reference_qubits[i], qs[j])
        c += cirq.ControlledGate(sub_gate=cirq.SWAP, num_controls=1).on(swap_qubit, reference_qubits[i], qs[j])
    c += cirq.H(swap_qubit)
    return c

In [11]:
state_qubits = 4
latent_qubits = 1
total_qubits = state_qubits + (state_qubits - latent_qubits)
qubits = [cirq.GridQubit(0, i) for i in range(total_qubits + 1 + state_qubits)]
print(len(qubits))

12


In [17]:
dataset = np.load("electron-photon.npz")
print(len(dataset["x_train"]))

100


In [18]:
train_circuits = [dataset["x_train"][i]  + dataset["y_train"][i] for i in range(100)]
test_circuits = [dataset["x_test"][i] + dataset["y_test"][i] for i in range(100)]